In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/test.csv
/kaggle/input/fake-news/train.csv
/kaggle/input/fake-news/submit.csv


In [2]:
import pandas as pd
import numpy as np 
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
import transformers
from tqdm import tqdm
import re
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', 200)

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [3]:
train = pd.read_csv('/kaggle/input/fake-news/train.csv')
display(train.head(5))

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It,Darrell Lucus,"House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy...",1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart",Daniel J. Flynn,"Ever get the feeling your life circles the roundabout rather than heads in a straight line toward the intended destination? [Hillary Clinton remains the big woman on campus in leafy, liberal Welle...",0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, 2016 \nThe tension between intelligence analysts and political policymakers has always been between honest assessments and desired results, with the l...",1
3,3,15 Civilians Killed In Single US Airstrike Have Been Identified,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstrike Have Been Identified The rate at which civilians are being killed by American airstrikes in Afghanistan is now higher than it was in 2014 when the...,1
4,4,Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery,Howard Portnoy,Print \nAn Iranian woman has been sentenced to six years in prison after Iran’s Revolutionary Guard searched her home and found a notebook that contained a fictional story she’d written about a wo...,1


In [31]:
test = pd.read_csv('/kaggle/input/fake-news/test.csv')
print(len(test))
display(test.head(5))

5200


,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse Strings, in Silicon Valley - The New York Times",David Streitfeld,"PALO ALTO, Calif. — After years of scorning the political process, Silicon Valley has leapt into the fray. The prospect of a President Donald J. Trump is pushing the tech community to move beyo..."
1,20801,Russian warships ready to strike terrorists near Aleppo,NaN,Russian warships ready to strike terrorists near Aleppo 08.11.2016 | Source: Source: Mil.ru Attack aircraft of the Russian aircraft carrier Admiral Kuznetsov get ready to strike terrorists' positi...
2,20802,"#NoDAPL: Native American Leaders Vow to Stay All Winter, File Lawsuit Against Police",Common Dreams,"Videos #NoDAPL: Native American Leaders Vow to Stay All Winter, File Lawsuit Against Police Amnesty International are sending a delegation of human rights observers to monitor the response of law ..."
3,20803,"Tim Tebow Will Attempt Another Comeback, This Time in Baseball - The New York Times",Daniel Victor,"If at first you don’t succeed, try a different sport. Tim Tebow, who was a Heisman quarterback at the University of Florida but was unable to hold an N. F. L. job, is pursuing a career in Major ..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,"42 mins ago 1 Views 0 Comments 0 Likes 'For the first time in history, we’re filming a panoramic video from the station. It means you’ll see everything we see here, with your own eyes. That’s to s..."


In [4]:
train['text'] = train['text'].fillna(train['title'])

In [29]:
test['text'] = test['text'].fillna(test['title'])

In [30]:
test.isna().sum() 

id          0
title     122
author    503
text        0
dtype: int64

In [5]:
train.fillna('',inplace = True)

In [32]:
test.fillna('',inplace = True)

In [6]:
def simplepreprocess(text):
    text = re.sub("[^a-zA-Z ]", "", text).strip()
    return text

In [7]:
train['preprocessed_text'] = train['text'].apply(simplepreprocess)

In [33]:
test['preprocessed_text'] = test['text'].apply(simplepreprocess)

In [8]:
train_data = train[['preprocessed_text','label']]

In [34]:
test_data = test[['preprocessed_text']]

In [9]:
train_df,eval_df = train_test_split(train_data,test_size = 0.05)

In [10]:
!pip install simpletransformers==0.32.3

     |████████████████████████████████| 187 kB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 43 kB 1.2 MB/s  eta 0:00:01
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16170 sha256=7b809eebb9711d09076221c5a4b7ed0898030a0c2692d2914d60ca50b9e0362c
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [11]:
!git clone https://github.com/NVIDIA/apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Cloning into 'apex'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 7456 (delta 0), reused 0 (delta 0), pack-reused 7455
Receiving objects: 100% (7456/7456), 13.91 MiB | 23.74 MiB/s, done.
Resolving deltas: 100% (5038/5038), done.
/opt/conda/lib/python3.7/site-packages/pip/_internal/commands/install.py:235: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
Using pip 20.2.4 from /opt/conda/lib/python3.7/site-packages/pip (python 3.7)
Non-user install because site-packages writeable
Created temporary directory: /tmp/pip-ephem-wheel-cache-fec0y2lv
Created temporary directory: /tmp/pip-req-tracker-zto7x84s
Initialized build tracking at /tmp/pip-req-tracker-zto7x84s
Created build tracker: /tmp/pip-req-tracker-zto7x84s
Entered build tracker: /tmp/pip-req-tracker-zto7x84s
Created temporary directory: /tmp/pip-install-1fs4py4k
Process

    running build_ext
    building 'apex_C' extension
    creating /tmp/pip-req-build-cie6la_u/build/temp.linux-x86_64-3.7
    creating /tmp/pip-req-build-cie6la_u/build/temp.linux-x86_64-3.7/csrc
    Emitting ninja build file /tmp/pip-req-build-cie6la_u/build/temp.linux-x86_64-3.7/build.ninja...
    Compiling objects...
    Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
    [1/1] c++ -MMD -MF /tmp/pip-req-build-cie6la_u/build/temp.linux-x86_64-3.7/csrc/flatten_unflatten.o.d -pthread -B /opt/conda/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/opt/conda/lib/python3.7/site-packages/torch/include -I/opt/conda/lib/python3.7/site-packages/torch/include/torch/csrc/api/include -I/opt/conda/lib/python3.7/site-packages/torch/include/TH -I/opt/conda/lib/python3.7/site-packages/torch/include/THC -I/opt/conda/include/python3.7m -c -c /tmp/pip-req-build-cie6la_u/csrc/flatte

    [4/11] /usr/local/cuda/bin/nvcc -I/opt/conda/lib/python3.7/site-packages/torch/include -I/opt/conda/lib/python3.7/site-packages/torch/include/torch/csrc/api/include -I/opt/conda/lib/python3.7/site-packages/torch/include/TH -I/opt/conda/lib/python3.7/site-packages/torch/include/THC -I/usr/local/cuda/include -I/opt/conda/include/python3.7m -c -c /tmp/pip-req-build-cie6la_u/csrc/multi_tensor_l2norm_kernel.cu -o /tmp/pip-req-build-cie6la_u/build/temp.linux-x86_64-3.7/csrc/multi_tensor_l2norm_kernel.o -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERSIONS__ -D__CUDA_NO_HALF2_OPERATORS__ --expt-relaxed-constexpr --compiler-options ''"'"'-fPIC'"'"'' -lineinfo -O3 --use_fast_math -DVERSION_GE_1_1 -DVERSION_GE_1_3 -DVERSION_GE_1_5 -DTORCH_API_INCLUDE_EXTENSION_H -DTORCH_EXTENSION_NAME=amp_C -D_GLIBCXX_USE_CXX11_ABI=0 -gencode=arch=compute_60,code=sm_60 -std=c++14
    [5/11] /usr/local/cuda/bin/nvcc -I/opt/conda/lib/python3.7/site-packages/torch/include -I/opt/conda/lib/python3.7/site-pac

    building 'syncbn' extension
    Emitting ninja build file /tmp/pip-req-build-cie6la_u/build/temp.linux-x86_64-3.7/build.ninja...
    Compiling objects...
    Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
    [1/2] /usr/local/cuda/bin/nvcc -I/opt/conda/lib/python3.7/site-packages/torch/include -I/opt/conda/lib/python3.7/site-packages/torch/include/torch/csrc/api/include -I/opt/conda/lib/python3.7/site-packages/torch/include/TH -I/opt/conda/lib/python3.7/site-packages/torch/include/THC -I/usr/local/cuda/include -I/opt/conda/include/python3.7m -c -c /tmp/pip-req-build-cie6la_u/csrc/welford.cu -o /tmp/pip-req-build-cie6la_u/build/temp.linux-x86_64-3.7/csrc/welford.o -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERSIONS__ -D__CUDA_NO_HALF2_OPERATORS__ --expt-relaxed-constexpr --compiler-options ''"'"'-fPIC'"'"'' -O3 -DVERSION_GE_1_1 -DVERSION_GE_1_3 -DVERSION_GE_1_5 -DTORCH_API_INCLUDE_EXTENSION_H -DTORCH_EXTENSION_

    /tmp/pip-req-build-cie6la_u/csrc/layer_norm_cuda.cpp:117:42: warning: ‘at::DeprecatedTypeProperties& at::Tensor::type() const’ is deprecated: Tensor.type() is deprecated. Instead use Tensor.options(), which in many cases (e.g. in a constructor) is a drop-in replacement. If you were using data from type(), that is now available from Tensor itself, so instead of tensor.type().scalar_type(), use tensor.scalar_type() instead and instead of tensor.type().backend() use tensor.device(). [-Wdeprecated-declarations]
     #define CHECK_CUDA(x) TORCH_CHECK(x.type().is_cuda(), #x " must be a CUDA tensor")
                                              ^
    /opt/conda/lib/python3.7/site-packages/torch/include/c10/macros/Macros.h:146:65: note: in definition of macro ‘C10_UNLIKELY’
     #define C10_UNLIKELY(expr)  (__builtin_expect(static_cast<bool>(expr), 0))
                                                                     ^~~~
    /opt/conda/lib/python3.7/site-packages/torch/include/c10/uti

...

    creating /opt/conda/lib/python3.7/site-packages/apex/amp
    copying build/lib.linux-x86_64-3.7/apex/amp/__init__.py -> /opt/conda/lib/python3.7/site-packages/apex/amp
    copying build/lib.linux-x86_64-3.7/apex/amp/compat.py -> /opt/conda/lib/python3.7/site-packages/apex/amp
    copying build/lib.linux-x86_64-3.7/apex/amp/rnn_compat.py -> /opt/conda/lib/python3.7/site-packages/apex/amp
    copying build/lib.linux-x86_64-3.7/apex/amp/_amp_state.py -> /opt/conda/lib/python3.7/site-packages/apex/amp
    copying build/lib.linux-x86_64-3.7/apex/amp/_initialize.py -> /opt/conda/lib/python3.7/site-packages/apex/amp
    copying build/lib.linux-x86_64-3.7/apex/amp/frontend.py -> /opt/conda/lib/python3.7/site-packages/apex/amp
    creating /opt/conda/lib/python3.7/site-packages/apex/amp/lists
    copying build/lib.linux-x86_64-3.7/apex/amp/lists/__init__.py -> /opt/conda/lib/python3.7/site-packages/apex/amp/lists
    copying build/lib.linux-x86_64-3.7/apex/amp/lists/tensor_overrides.py -> /

    byte-compiling /opt/conda/lib/python3.7/site-packages/apex/RNN/RNNBackend.py to RNNBackend.cpython-37.pyc
    byte-compiling /opt/conda/lib/python3.7/site-packages/apex/reparameterization/__init__.py to __init__.cpython-37.pyc
    byte-compiling /opt/conda/lib/python3.7/site-packages/apex/reparameterization/reparameterization.py to reparameterization.cpython-37.pyc
    byte-compiling /opt/conda/lib/python3.7/site-packages/apex/reparameterization/weight_norm.py to weight_norm.cpython-37.pyc
    byte-compiling /opt/conda/lib/python3.7/site-packages/apex/optimizers/__init__.py to __init__.cpython-37.pyc
    byte-compiling /opt/conda/lib/python3.7/site-packages/apex/optimizers/fused_adagrad.py to fused_adagrad.cpython-37.pyc
    byte-compiling /opt/conda/lib/python3.7/site-packages/apex/optimizers/fused_lamb.py to fused_lamb.cpython-37.pyc
    byte-compiling /opt/conda/lib/python3.7/site-packages/apex/optimizers/fused_novograd.py to fused_novograd.cpython-37.pyc
    byte-compiling /opt

    byte-compiling /opt/conda/lib/python3.7/site-packages/apex/contrib/optimizers/distributed_fused_adam_v3.py to distributed_fused_adam_v3.cpython-37.pyc
    byte-compiling /opt/conda/lib/python3.7/site-packages/apex/contrib/optimizers/fused_lamb.py to fused_lamb.cpython-37.pyc
    byte-compiling /opt/conda/lib/python3.7/site-packages/apex/contrib/optimizers/fp16_optimizer.py to fp16_optimizer.cpython-37.pyc
    byte-compiling /opt/conda/lib/python3.7/site-packages/apex/contrib/optimizers/distributed_fused_lamb.py to distributed_fused_lamb.cpython-37.pyc
    byte-compiling /opt/conda/lib/python3.7/site-packages/apex/contrib/optimizers/fused_sgd.py to fused_sgd.cpython-37.pyc
    byte-compiling /opt/conda/lib/python3.7/site-packages/apex/contrib/optimizers/fused_adam.py to fused_adam.cpython-37.pyc
    byte-compiling /opt/conda/lib/python3.7/site-packages/apex/contrib/optimizers/distributed_fused_adam_v2.py to distributed_fused_adam_v2.cpython-37.pyc
    byte-compiling /opt/conda/lib/p

  Found link https://files.pythonhosted.org/packages/36/fa/51ca4d57392e2f69397cd6e5af23da2a8d37884a605f9e3f2d3bfdc48397/pip-19.0.3.tar.gz#sha256=6e6f197a1abfb45118dbb878b5c859a0edbdd33fd250100bc015b67fded4b9f2 (from https://pypi.org/simple/pip/) (requires-python:>=2.7,!=3.0.*,!=3.1.*,!=3.2.*,!=3.3.*), version: 19.0.3
  Found link https://files.pythonhosted.org/packages/f9/fb/863012b13912709c13cf5cfdbfb304fa6c727659d6290438e1a88df9d848/pip-19.1-py2.py3-none-any.whl#sha256=8f59b6cf84584d7962d79fd1be7a8ec0eb198aa52ea864896551736b3614eee9 (from https://pypi.org/simple/pip/) (requires-python:>=2.7,!=3.0.*,!=3.1.*,!=3.2.*,!=3.3.*), version: 19.1
  Found link https://files.pythonhosted.org/packages/51/5f/802a04274843f634469ef299fcd273de4438386deb7b8681dd059f0ee3b7/pip-19.1.tar.gz#sha256=d9137cb543d8a4d73140a3282f6d777b2e786bb6abb8add3ac5b6539c82cd624 (from https://pypi.org/simple/pip/) (requires-python:>=2.7,!=3.0.*,!=3.1.*,!=3.2.*,!=3.3.*), version: 19.1
  Found link https://files.pythonhost

Removed build tracker: '/tmp/pip-req-tracker-zto7x84s'


In [15]:
from simpletransformers.classification import ClassificationModel


# Create a TransformerModel
model = ClassificationModel('bert', 'bert-base-cased', num_labels=2, 
                            args={'reprocess_input_data': True,
                                  'overwrite_output_dir': True,
                                  'use_early_stopping' : True,
                                  'early_stopping_patience' : 1,
                                  'evaluate_during_training' : True,
                                  'evaluate_during_training_steps' : 76,
                                  'evaluate_during_training_verbose' : True,
                                  'num_train_epochs': 5,
                                  'fp16' : False,
                                  'train_batch_size' : 128,
                                  'use_cached_eval_features' : True,
                                  'early_stopping_metric' : "eval_loss",
                                  'early_stopping_metric_minimize' : True,
                                  'early_stopping_delta' : 0.001
                                  
                                 }
                                  ,use_cuda=True)

In [16]:
model.train_model(train_df,eval_df = eval_df)

/opt/conda/lib/python3.7/site-packages/simpletransformers/classification/classification_model.py:263: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


Running loss: 0.247894

/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.113387

/opt/conda/lib/python3.7/site-packages/simpletransformers/classification/classification_model.py:683: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running loss: 0.009199


Running loss: 0.060903


Running loss: 0.083949


Running loss: 0.009926



In [25]:
import os
def listFiles(dir):
    print(dir)
    print(os.listdir(dir))

d = '/kaggle/working'
subdirs = [os.path.join(d, o) for o in os.listdir(d) 
                    if os.path.isdir(os.path.join(d,o))]
for subdir in subdirs:
    listFiles(subdir)

/kaggle/working/cache_dir
['cached_dev_bert_128_2_1040', 'cached_train_bert_128_2_19760']
/kaggle/working/BestSimpleTransformerModel.model
['optimizer.pt', 'eval_results.txt', 'training_args.bin', 'tokenizer_config.json', 'config.json', 'vocab.txt', 'model_args.json', 'special_tokens_map.json', 'scheduler.pt', 'pytorch_model.bin']
/kaggle/working/apex
['examples', 'tests', 'csrc', '.gitmodules', 'LICENSE', '.git', '.nojekyll', 'README.md', 'requirements_dev.txt', 'setup.py', 'apex', '.gitignore', 'requirements.txt', 'docs']
/kaggle/working/runs
['Nov13_08-57-09_e55a1a6a46c7']
/kaggle/working/outputs
['checkpoint-76', 'eval_results.txt', 'checkpoint-310-epoch-2', 'checkpoint-380', 'checkpoint-465-epoch-3', 'training_args.bin', 'tokenizer_config.json', 'checkpoint-456', 'config.json', 'vocab.txt', 'checkpoint-152', 'checkpoint-304', 'model_args.json', 'checkpoint-608', 'special_tokens_map.json', 'checkpoint-228', 'training_progress_scores.csv', 'pytorch_model.bin', 'checkpoint-532', 'che

In [18]:
#BestModel = ClassificationModel(
#    "bert", "/kaggle/working/outputs/best_model"
#)

OSError: Can't load config for '/kaggle/working/outputs/best_model'. Make sure that:

- '/kaggle/working/outputs/best_model' is a correct model identifier listed on 'https://huggingface.co/models'

- or '/kaggle/working/outputs/best_model' is the correct path to a directory containing a config.json file



In [26]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

In [27]:
print(result)
print(model_outputs)

{'mcc': 0.9788569749331367, 'tp': 506, 'tn': 523, 'fp': 4, 'fn': 7, 'eval_loss': 0.05861504099420227}
[[ 4.559593  -4.8692675]
 [ 4.4201574 -4.643036 ]
 [ 4.695879  -5.0054827]
 ...
 [-4.941856   5.0585423]
 [ 4.4427614 -4.7481847]
 [ 4.7476087 -4.900466 ]]


In [37]:
display(test_data.head(2))

,preprocessed_text
0,PALO ALTO Calif After years of scorning the political process Silicon Valley has leapt into the fray The prospect of a President Donald J Trump is pushing the tech community to move beyond its...
1,Russian warships ready to strike terrorists near Aleppo Source Source Milru Attack aircraft of the Russian aircraft carrier Admiral Kuznetsov get ready to strike terrorists positions in the vici...


In [39]:
prediction = model.predict(list(test_data.preprocessed_text))

In [41]:
print(prediction[0].shape)
print(prediction)

(5200,)
(array([0, 1, 1, ..., 0, 1, 0]), array([[ 4.754674 , -4.912527 ],
       [-4.603681 ,  4.5275164],
       [-5.0647583,  5.152321 ],
       ...,
       [ 4.553158 , -4.777217 ],
       [-5.015017 ,  5.2857814],
       [ 4.4910755, -4.7474227]], dtype=float32))


In [42]:
submit = pd.read_csv("/kaggle/input/fake-news/submit.csv")
print(len(submit))
display(submit.head(2))

5200


,id,label
0,20800,0
1,20801,1


In [43]:
submit['label'] = prediction[0]
display(submit.head(2))

,id,label
0,20800,0
1,20801,1


In [44]:
submit.to_csv('SimpleTransformerBaseBertSubmit.csv', index=False)